# Bibliotecas e Libs

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=33022de5c57893d6a678bb3f59be6d16612fc58b313e24aa79de7f3c230f8f54
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
# Importar as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

In [ ]:
# Criar uma sessão Spark
spark = SparkSession.builder.appName("Atividade01").getOrCreate()

# Parte 1

Desenvolva um notebook bem documentado em Python utilizando a biblioteca PySpark para apresentar as análises realizadas no trabalho do Hadoop

Para cada análise realizada, escreva a consulta em SQL no Spark e também utilize as funções vistas em aula
Acrescente novas análises para uma análise exploratória mais detalhada
Utilize funções de agregação (count, min, max, avg, sum, etc.)

### 1) A Diretora Camilla Prado solicitou uma pesquisa que informe todas as obras cadastradas no acervo ordenadas por data de publicação.

In [ ]:
# Ler fontes de dados

obras_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/obra_origem.CSV", encoding='latin1',header='infer',delimiter=';')
editoras_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/editora_origem.CSV", encoding='latin1',header='infer',delimiter=';')
autores_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/autor_origem.CSV", encoding='latin1',header='infer',delimiter=';')
spark_df = spark.createDataFrame(pd_df)


# Converter de pandas para um dataframe spark.
obras = spark.createDataFrame(obras_pd)
editoras = spark.createDataFrame(editoras_pd)
autores = spark.createDataFrame(autores_pd)


# Realizar o join das tabelas
obras_editoras = obras.join(editoras, on=["ID_Editora"], how="inner")
obras_autores = obras_editoras.join(autores, on=["ID_Autor"], how="inner")

# Selecionar as colunas desejadas
obras_autores_editoras = obras_autores.select(
    F.col("Titulo_Obra").alias("Obra"),
    F.col("Nome_Autor").alias("Autor"),
    F.col("Nome_Editora").alias("Editora"),
    F.col("Data_Publicação").alias("Data de Publicação"),
)

# Ordenar os dados pela data de publicação
obras_autores_editoras = obras_autores_editoras.orderBy("Data de Publicação")

# Exibir os resultados
obras_autores_editoras.show()

+--------------------+--------------------+-------+------------------+
|                Obra|               Autor|Editora|Data de Publicação|
+--------------------+--------------------+-------+------------------+
| Fortaleza de Sharpe|     Alexandre Dumas|   Casa|        01/01/2001|
|      Arte Da Guerra|George Raymond Ri...|Saraiva|        01/05/1913|
|    Filhos e Amantes|       Alberto Mussa|   Leya|        01/09/2005|
|     Falcão de Malta|   Casimiro de Abreu|Saraiva|        02/01/2010|
|      Finis Patriae |     Ariano Suassuna|   Casa|        04/03/2013|
|Triste Fim de Pol...|George Raymond Ri...|   Casa|        05/03/2001|
|Os Três Mosqueteiros|     Alexandre Dumas|  Aleph|        06/03/1953|
|   Diabo dos Números| Clive Staples Lewis|   Nova|        06/05/2013|
|      Furia dos Reis|George Raymond Ri...|   Leya|        06/08/2008|
|  Discurso do Método|     Ariano Suassuna|Moderna|        06/08/2012|
|         Vidas Secas|        Rick Riordan|Moderna|        09/07/2004|
|O Con

### 2)  O Governador vai doar duzentos livros para a Biblioteca, mas só irá doar se a biblioteca tiver menos de 300 obras. O Gerente Márcio Tales solicitou que fosse feita a contagem de quantas obras a Biblioteca possui atualmente.

In [ ]:
# Ler os dados da tabela
obras_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/obra_origem.CSV", encoding='latin1',header='infer',delimiter=';')
obras = spark.createDataFrame(obras_pd)
# Contar o número de títulos de obras distintos
total_obras = obras.select(F.countDistinct("Titulo_Obra").alias("Total de Obras"))

# Exibir os resultados
total_obras.show()

+--------------+
|Total de Obras|
+--------------+
|            29|
+--------------+



### 3) A Gerência solicitou uma pesquisa para saber quais datas ocorreram empréstimos de livros e a quantidade emprestada. A consulta deverá retornar apenas um registro para cada data.

In [ ]:
# Ler os dados da tabela
emprestimos_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/emprestimo_origem.CSV", encoding='latin1',header='infer',delimiter=';')
emprestimos = spark.createDataFrame(emprestimos_pd)
# Agrupar os empréstimos pela data de empréstimo
emprestimos_agrupados = emprestimos.groupBy("Data_Emprestimo")

# Contar o número de empréstimos por data
total_emprestimos = emprestimos_agrupados.agg(F.count("id_emprestimo").alias("Total_de_Emprestimos"))

# Ordenar os resultados pelo número de empréstimos em ordem decrescente
total_emprestimos = total_emprestimos.orderBy("Total_de_Emprestimos", ascending=False)

# Exibir os resultados
total_emprestimos.show()

+---------------+--------------------+
|Data_Emprestimo|Total_de_Emprestimos|
+---------------+--------------------+
|     07/07/2013|                   4|
|     20/08/2011|                   3|
|     19/08/2011|                   3|
|     25/06/2013|                   3|
|     31/01/2013|                   3|
|     28/12/2012|                   3|
|     18/08/2011|                   1|
|     11/07/2011|                   1|
|     01/04/2012|                   1|
|     25/03/2012|                   1|
|     29/03/2012|                   1|
|     31/03/2012|                   1|
|     27/03/2012|                   1|
|     15/08/2011|                   1|
|     09/06/2011|                   1|
|     26/09/2011|                   1|
+---------------+--------------------+



### 4) O Funcionário João Paulo Assistente de RH solicitou uma pesquisa que informasse todos os empréstimos que a Recepcionista Alice Meire fez no horário das 8hs as 9hs.


In [ ]:
# Ler os dados das tabelas
emprestimos_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/emprestimo_origem.CSV", encoding='latin1',header='infer',delimiter=';')
emprestimos = spark.createDataFrame(emprestimos_pd)

funcionarios_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/funcionario_origem.CSV", encoding='latin1',header='infer',delimiter=';')
funcionarios = spark.createDataFrame(funcionarios_pd)

obras_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/obra_origem.CSV", encoding='latin1',header='infer',delimiter=';')
obras = spark.createDataFrame(obras_pd)


# Realizar o join das tabelas
emprestimos_funcionarios = emprestimos.join(funcionarios, on=["ID_Funcionario"], how="inner")
emprestimos_obras = emprestimos_funcionarios.join(obras, on=["ID_Obra"], how="inner")

# Selecionar as colunas desejadas
emprestimos_filtrados = emprestimos_obras.select(
    F.col("Nome_Funcionario").alias("Nome do Funcionario"),
    F.col("Titulo_Obra").alias("Titulo da Obra"),
    F.col("Hora_Emprestimo").alias("Horário do Empréstimo"),
)

# Filtrar os empréstimos realizados pelo funcionário 8
emprestimos_filtrados = emprestimos_filtrados.filter(emprestimos_filtrados["Nome do Funcionario"] == "Alice Meire ")

# Filtrar os empréstimos realizados entre 8h00 e 9h00
emprestimos_filtrados = emprestimos_filtrados.filter(
    (emprestimos_filtrados["Horário do Empréstimo"] >= "8:00") & (
        emprestimos_filtrados["Horário do Empréstimo"] <= "9:00"
    )
)

# Exibir os resultados
emprestimos_filtrados.show()

+-------------------+----------------+---------------------+
|Nome do Funcionario|  Titulo da Obra|Horário do Empréstimo|
+-------------------+----------------+---------------------+
|       Alice Meire | Falcão de Malta|                 8:00|
|       Alice Meire |Força das Coisas|                 8:36|
|       Alice Meire |   Frankenstein |                 8:00|
+-------------------+----------------+---------------------+



### 5) - A Diretoria solicitou uma pesquisa sobre devolução de livros entre as datas 29/03/2012 a 02/02/2013.

In [ ]:
# Ler os DataFrames das tabelas dw_devolucao, dw_usuario e dw_obra
devolucao_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/devolucao_origem.CSV", encoding='latin1',header='infer',delimiter=';')
dw_devolucao = spark.createDataFrame(devolucao_pd)

usuario_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/usuario_origem.CSV", encoding='latin1',header='infer',delimiter=';')
dw_usuario = spark.createDataFrame(usuario_pd)

obras_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/obra_origem.CSV", encoding='latin1',header='infer',delimiter=';')
dw_obra = spark.createDataFrame(obras_pd)


# Realizar um join entre as tabelas dw_devolucao, dw_usuario e dw_obra
dw_devolucao_joined = dw_devolucao.join(dw_usuario, dw_devolucao["ID_Usuario"] == dw_usuario["ID_Usuario"], "inner")
dw_devolucao_joined = dw_devolucao_joined.join(dw_obra, dw_devolucao_joined["ID_Obra"] == dw_obra["ID_Obra"], "inner")

# Selecionar as colunas desejadas
dw_devolucao_filtered = dw_devolucao_joined.select(
    F.col("Nome_Usuario").alias("Nome Usuário"),
    F.col("Titulo_Obra").alias("Titulo da Obra"),
    F.col("Data_Devolucao"),
    F.col("Hora_Devolucao"),
    F.col("Multa_Atraso").alias("Multa por Atraso"),
)

# Filtrar as datas de devolução
dw_devolucao_filtered = dw_devolucao_filtered.filter(
    F.col("Data_Devolucao").between("29/03/2012", "02/02/2013")
)

# Ordenar o DataFrame pela data de devolução
dw_devolucao_ordered = dw_devolucao_filtered.orderBy(F.desc("Data_Devolucao"))

# Visualizar o DataFrame ordenado
dw_devolucao_ordered.show()

+------------+--------------+--------------+--------------+----------------+
|Nome Usuário|Titulo da Obra|Data_Devolucao|Hora_Devolucao|Multa por Atraso|
+------------+--------------+--------------+--------------+----------------+
+------------+--------------+--------------+--------------+----------------+



### 6) - A Gerência solicitou uma pesquisa para saber quais reservas de livros que foram feitas com data maior  ou igual a 18/08/2011.

In [ ]:
# Carregue os DataFrames correspondentes às tabelas
reserva_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/reserva_origem.csv", encoding='latin1',header='infer',delimiter=';')
dw_reserva = spark.createDataFrame(reserva_pd)

usuario_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/usuario_origem.CSV", encoding='latin1',header='infer',delimiter=';')
dw_usuario = spark.createDataFrame(usuario_pd)


obras_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/obra_origem.CSV", encoding='latin1',header='infer',delimiter=';')
dw_obra = spark.createDataFrame(obras_pd)

# Prefixe as colunas com o alias da tabela
dw_reserva = dw_reserva.alias("reserva")
dw_usuario = dw_usuario.alias("usuario")
dw_obra = dw_obra.alias("obra")

# Realize as junções e selecione as colunas desejadas
resultado = dw_reserva \
    .join(dw_usuario, F.col("reserva.ID_Usuario") == F.col("usuario.ID_Usuario")) \
    .join(dw_obra, F.col("reserva.ID_Obra") == F.col("obra.ID_Obra")) \
    .select(
        F.col("obra.Titulo_Obra").alias('Titulo da Obra'),
        F.col("reserva.Status_Livro").alias('Status da Reserva'),
        F.col("reserva.Data_Reserva").alias('Data da Reserva'),
        F.col("reserva.Hora_Reserva").alias('Hora da Reserva'),
        F.col("usuario.Nome_Usuario").alias('Reservado por')
    )

# Aplicar as condições do WHERE
resultado = resultado.filter(
    (F.col("Data da Reserva") == '18/08/2011') & (resultado['Status da Reserva'] == 'Reservado')
)

# Mostrar o resultado
resultado.show()


+-----------------+-----------------+---------------+---------------+-------------------+
|   Titulo da Obra|Status da Reserva|Data da Reserva|Hora da Reserva|      Reservado por|
+-----------------+-----------------+---------------+---------------+-------------------+
|Diabo dos Números|        Reservado|     18/08/2011|          15:00|Juliana Bento Souza|
|        Fluviais |        Reservado|     18/08/2011|          15:00|    Ernesto Coimbra|
+-----------------+-----------------+---------------+---------------+-------------------+



### 7) -  A área de RH solicitou uma pesquisa para saber quais devoluções de livros  foram feitas antes de 29/03/2012.

In [ ]:
# Ler os DataFrames das tabelas dw_devolucao e dw_obra
devolucao_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/devolucao_origem.CSV", encoding='latin1',header='infer',delimiter=';')
dw_devolucao = spark.createDataFrame(devolucao_pd)

obras_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/obra_origem.CSV", encoding='latin1',header='infer',delimiter=';')
dw_obra = spark.createDataFrame(obras_pd)


# Realizar um join entre as tabelas dw_devolucao e dw_obra
dw_devolucao_joined = dw_devolucao.join(dw_obra, "ID_Obra", "inner")

# Selecionar as colunas desejadas
dw_devolucao_filtered = dw_devolucao_joined.select(
    F.col("Titulo_Obra"),
    F.col("Data_Devolucao"),
    F.col("Hora_Devolucao"),
    F.col("Multa_Atraso"),
)

# Filtrar as datas de devolução
dw_devolucao_filtered = dw_devolucao_filtered.filter(
    F.col("Data_Devolucao") < "29/03/2012"
)

# Visualizar o DataFrame filtrado
dw_devolucao_filtered.show()

+--------------------+--------------+--------------+------------+
|         Titulo_Obra|Data_Devolucao|Hora_Devolucao|Multa_Atraso|
+--------------------+--------------+--------------+------------+
|O Conde de Monte ...|    03/04/2012|         10:00|           0|
|Triste Fim de Pol...|    13/07/2011|         13:00|           0|
|   Diabo dos Números|    21/08/2011|         14:55|           0|
|   Diabo dos Números|    08/04/2012|         11:11|           1|
|      Finis Patriae |    12/01/2013|         15:00|           1|
|Os Três Mosqueteiros|    26/08/2011|         10:00|           1|
|     Falcão de Malta|    11/06/2011|         18:00|           0|
|     Falcão de Malta|    17/08/2011|         14:55|           0|
|     Falcão de Malta|    28/09/2011|         14:29|           0|
|   Flores sem Fruto |    21/08/2011|         10:00|           0|
|           Fluviais |    20/08/2011|         12:01|           0|
|    Força das Coisas|    22/08/2011|         18:00|           0|
|       Fr

### 8) - A Gerência solicitou uma pesquisa para saber quais obras existem no acervo que são diferentes dos títulos das obras ‘O Conde de Monte Cristo’ e ‘Filhos e Amantes’  .

In [ ]:

# Ler o DataFrame da tabela dw_obra
obras_pd = pd.read_csv("https://raw.githubusercontent.com/viniciusperdigao/Infraestrutura-Hadoop-Infnet/main/bases/obra_origem.CSV", encoding='latin1',header='infer',delimiter=';')
dw_obra = spark.createDataFrame(obras_pd)


# Selecionar as colunas desejadas
dw_obra_filtered = dw_obra.select(
    F.col("Titulo_Obra"),
    F.col("Gênero"),
    F.col("Data_Publicação"),
)

# Filtrar as obras que não são "O Conde de Monte Cristo" ou "Filhos e Amantes"

dw_obra_filtered = dw_obra_filtered.filter( (F.col("Titulo_Obra") != "O Conde de Monte Cristo") & (F.col("Titulo_Obra") != "Filhos e Amantes"))

# Visualizar o DataFrame filtrado
dw_obra_filtered.show()

+--------------------+-----------+---------------+
|         Titulo_Obra|     Gênero|Data_Publicação|
+--------------------+-----------+---------------+
|Tratado de Confissom|  Filosofia|     31/12/2010|
|Triste Fim de Pol...|   Política|     05/03/2001|
|Tratado da Nature...|    Romance|     14/05/2002|
|Farsa de Inês Per...|  Religioso|     25/04/1986|
|        Filho Nativo|      Poema|     12/06/2004|
|     Jogo Dos Tronos|     Ficção|     26/08/2001|
|   Diabo dos Números|     Terror|     31/08/1981|
|      Furia dos Reis|Estrangeiro|     06/08/2008|
|      Finis Patriae |   Política|     04/03/2013|
|     Finnegans Wake |    Romance|     30/09/2013|
|Os Três Mosqueteiros|    Romance|     06/03/1953|
|     Falcão de Malta| Auto Ajuda|     02/01/2010|
|         Vidas Secas|  Filosofia|     09/07/2004|
|   Flores sem Fruto |   Política|     15/08/2001|
|  Deixados para Trás|   Política|     12/09/1931|
|     Deus das Moscas|    Romance|     25/09/2003|
|    Senhor dos Aneis|     Ficç

# Parte 2

Desenvolva um notebook bem documentado em Python utilizando a biblioteca PySpark para treinar um modelo de aprendizado de máquina

## Exemplo de análise preditiva em dados de inspeção de alimentos

Contexto: Usaremos o  Spark para executar uma análise preditiva sobre os dados de inspeção de alimentos (Food_Inspections1.csv). Dados adquiridos por meio do portal de dados da cidade de Chicago. Esse conjunto de dados contém informações sobre inspeções de estabelecimentos de alimentos realizadas em Chicago. Incluindo informações sobre cada estabelecimento de alimentos inspecionado, as violações encontradas (se houver) e os resultados da inspeção.

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *

In [10]:
import pyspark


In [6]:
# Criar uma sessão Spark
spark = SparkSession.builder.appName("Atividade02").getOrCreate()

In [ ]:
https://raw.githubusercontent.com/jpvelez/restaurant_inspection_analysis/master/data/food_inspections.csv

Vamos criar um dataframe com os dados do food inspections lendo uma base do github.

In [7]:
df_pd = pd.read_csv("https://raw.githubusercontent.com/jpvelez/restaurant_inspection_analysis/master/data/food_inspections.csv", encoding='utf-8',header='infer')
df = spark.createDataFrame(df_pd)

<ipython-input-7-ff60f51c67c8>:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pd = pd.read_csv("https://raw.githubusercontent.com/jpvelez/restaurant_inspection_analysis/master/data/food_inspections.csv", encoding='utf-8',header='infer')


## Entendendo os dados

Vamos começar a ter uma ideia do que o nosso conjunto de dados contém.

In [25]:
df.show(10)

+-------------+--------------------+--------------------+---------+------------------+---------------+--------------------+-------+-----+-------+---------------+--------------------+-------+--------------------+------------+------------+-----------+------------+--------------------+
|Inspection ID|            DBA Name|            AKA Name|License #|     Facility Type|           Risk|             Address|   City|State|    Zip|Inspection Date|     Inspection Type|Results|          Violations|X Coordinate|Y Coordinate|   Latitude|   Longitude|            Location|
+-------------+--------------------+--------------------+---------+------------------+---------------+--------------------+-------+-----+-------+---------------+--------------------+-------+--------------------+------------+------------+-----------+------------+--------------------+
|      1229917|     ACAPULCO ALEGRE|     ACAPULCO ALEGRE|2162511.0|        Restaurant|  Risk 1 (High)|5306 W FULLERTON ...|CHICAGO|   IL|60639.0|   

Vamos checar valores distintos de resultados:

In [11]:
df.select('results').distinct().show()

+--------------------+
|             results|
+--------------------+
|                Fail|
|            No Entry|
|Business Not Located|
|  Pass w/ Conditions|
|     Out of Business|
|                Pass|
+--------------------+



Vamos visualizar a distribuição desses resultados:

In [16]:
df.createOrReplaceTempView('CountResults')
resultado = spark.sql("SELECT COUNT(results) AS cnt, results FROM CountResults GROUP BY results")
resultado.show()

+-----+--------------------+
|  cnt|             results|
+-----+--------------------+
|10624|                Fail|
|   11|            No Entry|
|   33|Business Not Located|
| 3587|  Pass w/ Conditions|
| 2777|     Out of Business|
|28258|                Pass|
+-----+--------------------+



Para prever um resultado de inspeção de alimentos, você precisa desenvolver um modelo com base nas violações. Como a regressão logística é um método de classificação binária, faz sentido agrupar os dados de resultado em duas categorias: Reprovado e Aprovado:

* Aprovado

  * Aprovado
  * Aprovado c/ condições

* Falha

  * Falha

* Descartar
  * Negócios não localizados
  * Fora de negócio
  
Dados com os outros resultados (“Negócios não localizados” ou “Fora de negócio”) não são úteis, e formam uma porcentagem bem pequena dos resultados de qualquer maneira.

O código a seguir é utilizado para converter nosso dataframe existente (df) em um novo dataframe em que cada inspeção é representada como um par de violações de rótulo. Nesse caso, um rótulo de 0.0 representa uma falha, um rótulo de 1.0 representa um sucesso e um rótulo de -1.0 representa alguns resultados diferentes desses dois.

In [27]:
def labelForResults(s):
    if s == 'Fail':
        return 0.0
    elif s == 'Pass w/ Conditions' or s == 'Pass':
        return 1.0
    else:
        return -1.0
label = UserDefinedFunction(labelForResults, DoubleType())
labeledData = df.select(label(df.Results).alias('label'), df.Violations).where('label >= 0')

In [22]:
labeledData.show()

+-----+--------------------+
|label|          Violations|
+-----+--------------------+
|  0.0|18. NO EVIDENCE O...|
|  1.0|                 NaN|
|  0.0|18. NO EVIDENCE O...|
|  1.0|41. PREMISES MAIN...|
|  1.0|34. FLOORS: CONST...|
|  0.0|2. FACILITIES TO ...|
|  1.0|35. WALLS, CEILIN...|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
|  1.0|                 NaN|
+-----+--------------------+
only showing top 20 rows



In [23]:
labeledData.take(1)

[Row(label=0.0, Violations='18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS\nComments:NO PEST CONTROL LOG BOOK ON SITE AT TIME OF INSPECTION, MUST PROVIDE, PREMISES OPEN FOR BUSINESS, AT THIS TIME,  SERIOUS CITATION ISSUED: 7-38-020 \n33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS\nComments:MUST REPLACE/CLEAN STAINED CUTTING IN PREP AREA \n35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS\nComments:MUST PROVIDE MISSING CEILING TILE REAR KITCHEN AREA, AND REPLACE STAINED CEILING TILE WHERE NEEDED \n37. TOILET ROOM DOORS SELF CLOSING: DRESSING ROOMS WITH LOCKERS PROVIDED: COMPLETE SEPARATION FROM LIVING/SLEEPING QUARTERS\nComments:SELF-CLOSING DEVICE NEEDED AT BATHROOM DOOR \n38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQUIRED: PLUMBING: INSTALLED AND MAINTAINED\nComments:MUST PROVIDE THR

# Criando um modelo de regressão logística do dataframe de entrada

A tarefa final é converter os dados rotulados. Converta os dados para um formato que possa ser analisado pela regressão logística. A entrada em um algoritmo de regressão logística precisa ser um conjunto de pares de vetor de recurso de rótulo. Em que o “vetor de recurso” é um vetor de números que representa o ponto de entrada. Portanto, você precisa converter a coluna “violações”, que é semiestruturada e contém vários comentários em texto livre. Converta a coluna para uma matriz de números reais que um computador entenderia facilmente.

Uma abordagem de aprendizado de máquina padrão para o processamento de linguagem natural é atribuir um “índice” a cada palavra distinta. Depois, passe um vetor para o algoritmo de aprendizado de máquina. Faça isso de modo que o valor de cada índice contenha a frequência relativa dessa palavra na cadeia de texto.

A MLlib proporciona uma forma fácil de executar essa operação. Primeiro, crie tokens de cada cadeia de caracteres de violações para obter as palavras individuais em cada cadeia de caracteres. Em seguida, use um HashingTF para converter cada conjunto de tokens em um vetor de recurso que pode ser passado para o algoritmo de regressão logística para construir um modelo. Você realiza todas essas etapas em sequência, usando um "pipeline".

In [34]:
tokenizer = Tokenizer(inputCol="Violations", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

model = pipeline.fit(labeledData)

Agora podemso usar o modelo criado  para prever quais serão os resultados das novas inspeções. As previsões são baseadas nas violações que foram observadas.
Esse modelo foi treinado pelo conjunto de dados Food_Inspections.csv.
Podemos também usar um segundo conjunto de dados, Food_Inspections2.csv, para avaliar a força deste modelo nos dados novos.


In [ ]:
testData = sc.textFile('wasbs:///HdiSamples/HdiSamples/FoodInspectionData/Food_Inspections2.csv')\
            .map(csvParse) \
            .map(lambda l: (int(l[0]), l[1], l[12], l[13]))
testDf = spark.createDataFrame(testData, schema).where("results = 'Fail' OR results = 'Pass' OR results = 'Pass w/ Conditions'")
predictionsDf = model.transform(testDf)
predictionsDf.registerTempTable('Predictions')
predictionsDf.columns

In [ ]:
predictionsDf.take(1)

O método model.transform() aplica a mesma transformação para novos dados com o mesmo esquema e chega a uma previsão de como classificar os dados. Você pode realizar algumas estatísticas simples para ter uma ideia de quão precisas foram as previsões:

In [ ]:
numSuccesses = predictionsDf.where("""(prediction = 0 AND results = 'Fail') OR
                                        (prediction = 1 AND (results = 'Pass' OR
                                                            results = 'Pass w/ Conditions'))""").count()
numInspections = predictionsDf.count()

print "There were", numInspections, "inspections and there were", numSuccesses, "successful predictions"
print "This is a", str((float(numSuccesses) / float(numInspections)) * 100) + "%", "success rate"